In [7]:
import os
import re
import openai
from openai import OpenAI
import time
import json
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')
GPT_MODEL = "gpt-4-1106-preview"
client = OpenAI()

def submit_message_and_create_run(assistant_id, prompt):
    thread = client.beta.threads.create()
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=prompt)
    return client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id), thread

def wait_on_run_and_get_response(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        time.sleep(0.5)
    messages = client.beta.threads.messages.list(thread_id=thread.id, order="asc")
    return [m.content[0].text.value for m in messages if m.role == 'assistant']

assistant = client.beta.assistants.create(
    name="Story Writer",
    instructions="You are an exceptionally creative story writer",
    model=GPT_MODEL,
)

ASSISTANT_ID = assistant.id

def ask_gpt3(prompt, ASSISTANT_ID = assistant.id, max_retries=3, delay=2):
    retries = 0
    run, thread = submit_message_and_create_run(ASSISTANT_ID, prompt)
    response = wait_on_run_and_get_response(run, thread)
    return response

def get_chars(input):
    messages = f"To solve this input, we need a list of characters and descriptions, both major and minor, who will have key parts to play. Help me write this please. {input}."
    chars = ask_gpt3(messages)
    return chars

def get_story(input, chars):
    messages = f"From the given prompt, generate a 6-part structure for a story, where each part has a 1 sentence description of what exactly happens in that part. Please start each part with Part X: {input}. The key characters for the overall story are {chars}"
    response = ask_gpt3(messages)
    print(f"The structure of the story as returned: \n {response}")
    storyline_parts = re.split(r'Part \d+:', response)[1:]
    return [part.strip() for part in storyline_parts]

def write_chapters(chars, storyline_parts):
    with open('story.txt', 'w', encoding='utf-8') as story_file:
        for index, part in enumerate(storyline_parts):
            print(f"{index}: {part}")
        for index, chapter_summary in enumerate(storyline_parts):
            messages = f"For this chapter, write the story and dialogue to explore it fully and bring the chapter to life. {chapter_summary}. The key characters for the overall story are {chars}"
            chapter_content = ask_gpt3(messages)

            # Save chapter to file
            story_file.write(f'Chapter {index + 1}\n\n')
            story_file.write(chapter_content)
            story_file.write('\n\n')

if __name__ == "__main__":
    input_text = "Write a short story in the style of Geoff Dyer about a spaceship voyage through the universe where a robot is just trying to live life."
    chars = get_chars(input_text)
    storyline = get_story(input_text, chars)
    write_chapters(chars, storyline)
    print("Story has been written to 'story.txt'")

TypeError: expected string or bytes-like object, got 'list'